# Homomesy, Homometery & Parking Funtions of all Order

## Main Proof: Homomesic statistics for Rotate front-to-back \- From Parking Functions Project

The front-to-back rotation of a parking function $\sigma$ composed of $(\sigma_1, \sigma_2, \sigma_3, \ldots, \sigma_n)$ is the parking function $\tau$ with $\tau(\sigma) = (\sigma_2, \sigma_3, \ldots, \sigma_n, \sigma_1)$.

E.g. $\sigma = [1~2~3]$, $\tau = [2~3~1]$.

### Sage imports

In [19]:
# sage imports
# import Parking Functions from sage
from sage.combinat.parking_functions import *

# Using Findstat; front-to-back is map 293
from sage.databases.findstat import *
findstat()._allow_execution = True

# To use methods connected to orbits
from sage.combinat.cyclic_sieving_phenomenon import *

### Exisiting bijections and statistics for parking functions in FindStat's database

In [20]:
# All bijective maps from parking functions to parking functions
pf_bijection_ids = [293, 298, 299, 300, 301, 303, 304, 320, 52]
pf_bijections = [findmap(id) for id in pf_bijection_ids]

print("Parking Function Bijections: ")
for pf_bijection in pf_bijections:
    print(pf_bijection)

# All parking functions statistics
pf_stat_ids = [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937, 1946, 1947]
pf_stat_ids_wo_errors = [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937] # last two new codes (End of May 2024) don't run properly
pf_stats = [findstat(id) for id in pf_stat_ids]
pf_stats_wo_errors = [findstat(id) for id in pf_stat_ids_wo_errors]

print("\nParking Function Statistics: ")
for pf_stat in pf_stats:
    print(pf_stat)

Parking Function Bijections: 
Mp00293: rotate front-to-back
Mp00298: repeats to leading
Mp00299: ones to leading
Mp00300: leading to ones
Mp00301: leading to repeats
Mp00303: complement
Mp00304: rotate back-to-front
Mp00320: reverse
Mp00052: to non-decreasing parking function

Parking Function Statistics: 
St000135: The number of lucky cars of the parking function.
St000136: The dinv of a parking function.
St000165: The sum of the entries of a parking function.
St000188: The area of the Dyck path corresponding to a parking function and the total displacement of a parking function.
St000194: The number of primary dinversion pairs of a labelled dyck path corresponding to a parking function.
St000195: The number of secondary dinversion pairs of the dyck path corresponding to a parking function.
St000540: The sum of the entries of a parking function minus its length.
St000942: The number of critical left to right maxima of the parking functions.
St000943: The number of spots the most unluc

### Generating classical parking functions and other superset pfs

In [23]:
# All generators for orginal parking functions
def k_naples_l_interval_m_cars_n_spots_parking_functions(k, l, m, n):
    # Define the basic variables descriptively 
    max_back_steps = k # k-naples
    max_forward_steps = l # l-interval 
    total_cars = m # m-cars
    total_spots = n # n-spots

    street = [0] * (total_spots + 1) # Initialize a street with all spots empty
    preferences, parking_functions = [], [] # Initialize lists for preferences and valid parking functions

    def helper(cars_parked): 
        if cars_parked == total_cars: # Base case: all cars are parked
            parking_functions.append(preferences[:]) # Add a copy of the current preference list to parking functions
            return 1
            
        count = 0
        for preference_value in range(total_spots + 1):  # Iterate through all possible preference values
            spot = preference_value
            while street[spot] != 0 and preference_value - spot < max_back_steps and spot > 0:
                spot -= 1  # Move backward if the spot is occupied and within max_back_steps
            while street[spot] != 0 and spot - preference_value <= max_forward_steps:
                spot += 1  # Move forward if the spot is occupied and within max_forward_steps
                if spot > total_spots:  # Break if moving beyond the last spot
                    break
            if spot == total_spots or spot - preference_value > max_forward_steps:
                continue  # Skip if the car can't be parked within the constraints

            street[spot] = 1  # Mark the spot as occupied
            preferences.append(preference_value + 1)  # Add the preference value to the list
            count += helper(cars_parked + 1)  # Recur to park the next car
            street[spot] = 0  # Backtrack: unmark the spot
            preferences.pop()  # Backtrack: remove the preference value
        return count

    helper(0) # Start the recursive parking with 0 cars parked
    return parking_functions # Return all valid parking functions

def k_naples_l_interval_parking_functions(k, l, n):  # m = n 
    return k_naples_l_interval_m_cars_n_spots_parking_functions(k, l, n, n)

def l_interval_m_cars_n_spots_parking_functions(l, m, n): # no k
    return k_naples_l_interval_m_cars_n_spots_parking_functions(0, l, m, n)

def k_naples_m_cars_n_spots_parking_functions(k, m, n): # no l
    return k_naples_l_interval_m_cars_n_spots_parking_functions(k, n, m, n)

def l_interval_parking_functions(l, n): # no k and m = n 
    return k_naples_l_interval_m_cars_n_spots_parking_functions(0, l, n, n)

def k_naples_parking_functions(k, n): # no l and m = n 
    return k_naples_l_interval_m_cars_n_spots_parking_functions(k, n, n, n)

def basic_parking_functions(n): # classical parking functions  
    return k_naples_l_interval_m_cars_n_spots_parking_functions(0, n, n, n)

def basic_parking_functions_with_m_cars_and_n_spots(m, n): # classical, but m < n
    return k_naples_l_interval_m_cars_n_spots_parking_functions(0, n, m, n)

def unit_interval_parking_functions(n): # k = 1, l = 1, and m = n
    return k_naples_l_interval_m_cars_n_spots_parking_functions(1, 1, n, n)

def one_interval_parking_functions(n): # no k, l = 1, and m = n
    return k_naples_l_interval_m_cars_n_spots_parking_functions(0, 1, n, n)

### Homomesy test functions for parking function sets

In [24]:
# tests all bijections and statistics for homomesy | Sage Compliant | Cite Elder's et al. Papers
def run_homomesic_on_PFs(pfs): 
    findstat()._allow_execution = True
    for function in FindStatMaps(domain='Cc0023', codomain='Cc0023'):
        if function.properties_raw().find('bijective') >= 0:
            pfset = [ParkingFunction(pf) for pf in pfs]  # Convert each list to a ParkingFunction object
            F = DiscreteDynamicalSystem(pfset, function)
            for stat in pf_stats_wo_errors: # all PF stats on FindState minus the ones without codes
                if F.is_homomesic(stat): # prints map and stat that is homomesic under the set of given Parking Functions
                    print(function.id(), stat.id())

# tests homomesy for the one case it works for on classical parking functions
def test_homomesy_for_unique_case(pfs): 
    findstat()._allow_execution = True
    stat, bij_map= findstat(1903), findmap(293) # test for a specific map and stat for given sets of pfs 
    pfset = [ParkingFunction(pf) for pf in pfs]  # Convert each list to a ParkingFunction object
    
    F = DiscreteDynamicalSystem(pfset, bij_map)
    if F.is_homomesic(stat):
        print(bij_map.id(), stat.id())

### Subsets of classical parking functions

In [27]:
# Prime parking functions
def are_prime(pfs, n):
    result = []
    for pf in pfs:
        pf_list = pf[:]  # Create a copy of the list to avoid modifying the original list
        try:
            one = pf_list.index(1)
            del pf_list[one]
        except ValueError:
            continue  # Skip if there is no 1 in the list

        if is_prime(pf_list, n):
            result.append(pf)
    return result


def is_prime(pf, n):
    return tuple(pf) in {tuple(x) for x in basic_parking_functions(n - 1)}

# Newly discovered 'Picky' parking functions
def are_picky(pfs):
    result = []
    for pf in pfs:
        if is_picky(pf):
            result.append(pf)
    return result

def is_picky(pf):
    n = len(pf)
    distinct_ranks = sorted(set(pf))
    return distinct_ranks == list(range(1, len(distinct_ranks) + 1)) and all(1 <= x <= n for x in pf)

# Fubini rankings; subset of classical parking functions
def are_fubini_rankings(pfset):
    result = []
    for pf in pfset:
        if is_a_fubini_ranking(pf):
            result.append(pf)
    return result
    
def is_a_fubini_ranking(pf):
    sorted_pf = sorted(pf)
    current_value = sorted_pf[0]
    if current_value != 1:
        return False
    count = 0
    
    for value in sorted_pf:
        if value == current_value:
            count += 1
        else:
            if value != current_value + count:
                return False
            current_value = value
            count = 1
    return True

# Primary parking functions (find source online?)
def is_primary(pfset):
    result = []
    for pf in pfset:
        primary = True
        for idx, car in enumerate(pf):
            if car > idx + 1:
                primary = False
                break
        if primary:
            result.append(pf)
    return result

# Unit Fubini (intersection of fubini and unit pfs)
def generate_unit_fubini(n):
    pfset = unit_interval_parking_functions(n)
        
    basic = basic_parking_functions(n)
    pfset_of_sage_pfs = [ParkingFunction(pf) for pf in basic]  # Convert each list to a ParkingFunction object
    sagepfset = are_fubini_rankings(pfset_of_sage_pfs)
      
    intersection = list_of_lists_intersection(sagepfset, pfset)
    return intersection 

def list_of_lists_intersection(list1, list2):
    # Convert inner lists to tuples so they can be compared as sets
    set1 = set(tuple(x) for x in list1)
    set2 = set(tuple(x) for x in list2)
    
    # Find the intersection
    intersection = set1.intersection(set2)
    
    # Convert the tuples back to lists
    return [list(x) for x in intersection]

In [26]:
# Friendship parking functions (have cyrus double check?; just permutations?)
from itertools import combinations

def generate_friendship_pfs(n):
    all_graphs = generate_all_graphs(n)
    basic_pfs = basic_parking_functions(n)
    
    result = {}

    for edges in all_graphs:
        for pf in basic_pfs:
            if are_valid(edges, pf):
                if tuple(edges) not in result:
                    result[tuple(edges)] = []
                result[tuple(edges)].append(pf)
    return result

def are_valid(edges, pf):
    parking_lot = [None] * len(pf) # Initialize parking lot with None
    zipped_pf = list(zip(pf, range(1, len(pf) + 1)))
    
    for spot, car in zipped_pf:
        result = can_park(parking_lot, spot, car, edges)
        if not result:
            spot += 1
            result = can_park(parking_lot, spot, car, edges)
            if spot == len(pf) and not result:
                return False
    return True

def can_park(parking_lot, spot, car, edges): 
    if spot < 0 or spot >= len(parking_lot):
        return False

    if parking_lot[spot] is not None:
        spot += 1
        if spot >= len(parking_lot):
            return False

    left_ok = (spot == 0 or parking_lot[spot - 1] is None or friendly(edges, car, parking_lot[spot - 1]))
    right_ok = (spot == len(parking_lot) - 1 or parking_lot[spot + 1] is None or friendly(edges, car, parking_lot[spot + 1]))

    if left_ok and right_ok:
        parking_lot[spot] = car
        return True

    return False

def friendly(edges, car1, car2):
    if car2 == None:
        return True
    return (car1, car2) in edges or (car2, car1) in edges

def generate_all_graphs(n):
    # Generate all possible edges
    vertices = list(range(1, n + 1))
    possible_edges = list(combinations(vertices, 2))
    
    all_graphs = []
    
    # Generate all possible subsets of edges
    for r in range(len(possible_edges) + 1):
        for subset in combinations(possible_edges, r):
            all_graphs.append(list(subset))
    
    return all_graphs

# pfset = generate_friendship_pfs(4)
# for graph, pfs in pfset.items():
#     print("Graph:", graph)
#     print(len(pfs))
#     print()  # Add a newline for better readability

### Functions to test homometry

In [57]:
from collections import defaultdict

# Finds homometry triples given a set
def find_homometricty(pfs):
    result = []
    for bijection in [293, 298, 299, 300, 301, 303, 304, 320, 52]:
        bij = findmap(bijection)
        for statistic in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]:
            orbits = orbit_decomposition(pfs, bij)
            stat = findstat(statistic)
            grouped_averages = defaultdict(list)
            
            for orbit in orbits:
                orb_stat_avg = round(sum([stat(p) for p in orbit]) / float(len(orbit)), 2)
                orbit_length = len(orbit)
                grouped_averages[orbit_length].append((orbit, orb_stat_avg))
            
            num_different_orbit_lengths = len(grouped_averages)

            # Flag to check if all avg_values in all orbit_length buckets are the same
            all_same = True
            for orbit_length, stats in grouped_averages.items():
                avg_values = [stat[1] for stat in stats]
                if len(set(avg_values)) != 1:
                    all_same = False
                    break

            if all_same:
                print(f"\nBijection: {bijection}, Statistic: {statistic}, Num of Diff. Orbit Length: {num_different_orbit_lengths}")
                for orbit_length, stats in grouped_averages.items():
                    print(f"Orbit Length: {orbit_length}")
                    for orbit, avg in stats:
                        print(f"  Orbit: {orbit}, Average: {avg}")

# Verify if triple is homometric
def is_homometric_triple(A, B, X): # A must be a set of pfs, B must be the findmap(bijection) and X must be the findstat(stat)
    orbits = orbit_decomposition(A, B)
    averages, grouped_averages= [], defaultdict(list)
    
    for orbit in orbits:
        orb_stat_avg = round(sum(stat(p) for p in orbit) / float(len(orbit)), 2)
        orbit_length = len(orbit)
        averages.append((orbit_length, orb_stat_avg))
        grouped_averages[orbit_length].append(orb_stat_avg)
    
    # Process each group to check if all values are the same
    for orbit_length, stats in grouped_averages.items():
        if len(set(stats)) != 1:
            return False
    return True

### Tests for homomesy and homometry on classical parking functions and their subsets (+ sequences of cardinality)

#### Classical Parking Functions

In [44]:
print("Seq. of Sizes of Classical Parking Functions:")

result = []
for n in range(1, 8):
    pfset = basic_parking_functions(n)
    result.append(len(pfset))
print(result)

Seq. of Sizes of Classical Parking Functions:
[1, 3, 16, 125, 1296, 16807, 262144]


In [45]:
for n in range(2, 6):
    print("Classical Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    result = run_homomesic_on_PFs(pfset)
    if result is not None:
        print(result)
    print("\n")

Classical Parking Functions: n = 2
293 195
293 1903
298 195
299 195
300 195
301 195
304 195
304 1903


Classical Parking Functions: n = 3
293 1903
304 1903


Classical Parking Functions: n = 4
293 1903
304 1903


Classical Parking Functions: n = 5
293 1903
304 1903




In [46]:
# classical parking functions at n = 3; shows mathcing values (homomesy, as we know there are no homometric classical pf triples)
for n in range(2, 6):
    print("\nClassical Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    sagePFs = [ParkingFunction(pf) for pf in pfset]
    print(find_homometricty(sagePFs))


Classical Parking Functions: n = 2

Bijection: 293, Statistic: 135, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 1.0
Orbit Length: 2
  Orbit: [[1, 2], [2, 1]], Average: 2.0

Bijection: 293, Statistic: 136, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 0.0
Orbit Length: 2
  Orbit: [[1, 2], [2, 1]], Average: 0.5

Bijection: 293, Statistic: 165, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 2.0
Orbit Length: 2
  Orbit: [[1, 2], [2, 1]], Average: 3.0

Bijection: 293, Statistic: 188, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 1.0
Orbit Length: 2
  Orbit: [[1, 2], [2, 1]], Average: 0.0

Bijection: 293, Statistic: 194, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 0.0
Orbit Length: 2
  Orbit: [[1, 2], [2, 1]], Average: 0.5

Bijection: 293, Statistic: 195, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 1]], Average: 0.0
Orbit Length: 2
  Orbit:

From our first expierment, we found that the only map that showed homomesy with the set of classical parking functions was the rotate front-to-back map (and also it's inverse) and the number of fixed points statistics.

#### 'Picky' Parking Functions


In [47]:
print("Seq. of Sizes of 'Picky' Parking Functions:")

result = []
for n in range(1, 8):
    pfset = basic_parking_functions(n)
    picky_set = are_picky(pfset)
    result.append(len(picky_set))
print(result)

Seq. of Sizes of 'Picky' Parking Functions:
[1, 3, 13, 75, 541, 4683, 47293]


In [48]:
for n in range(2, 5):
    print("'Picky' Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    picky_set = are_picky(pfset)
    result = run_homomesic_on_PFs(picky_set)
    if result is not None:
        print(result)
    print("\n")

'Picky' Parking Functions: n = 2
293 195
293 1903
298 195
299 195
300 195
301 195
304 195
304 1903


'Picky' Parking Functions: n = 3
293 195
293 1903
298 195
301 195
304 195
304 1903


'Picky' Parking Functions: n = 4
293 195
293 1903
304 195
304 1903




In [49]:
for n in range(3, 6):
    print("\n'Picky' Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    picky_set = are_picky(pfset)
    sagePFs = [ParkingFunction(pf) for pf in picky_set]
    print(find_homometricty(sagePFs))


'Picky' Parking Functions: n = 3

Bijection: 293, Statistic: 195, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 0.0
  Orbit: [[1, 3, 2], [3, 2, 1], [2, 1, 3]], Average: 0.0
  Orbit: [[1, 2, 3], [2, 3, 1], [3, 1, 2]], Average: 0.0
  Orbit: [[1, 2, 2], [2, 2, 1], [2, 1, 2]], Average: 0.0
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 1903, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 1.0
  Orbit: [[1, 3, 2], [3, 2, 1], [2, 1, 3]], Average: 1.0
  Orbit: [[1, 2, 3], [2, 3, 1], [3, 1, 2]], Average: 1.0
  Orbit: [[1, 2, 2], [2, 2, 1], [2, 1, 2]], Average: 1.0
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 1.0

Bijection: 298, Statistic: 195, Num of Diff. Orbit Length: 3
Orbit Length: 4
  Orbit: [[1, 2, 1], [2, 3, 1], [1, 2, 3], [1, 2, 2]], Average: 0.0
  Orbit: [[2, 1, 1], [2, 1, 2], [2, 1, 3], [3, 2, 1]], Average: 0.0
Orbit Length: 2
  Orbit: [[1, 3, 2], 

In this new subset of classical parking functions which we found trying to write functions for fubini rankings (anotehr subset of PFs), we found that the same rotate front-to-back map exhibits homomesy, however, it does for Stat1903, the previous statistic, but it also exhibits homomesy for Stat195, the number of secondary dinversion pairs of the dyck path corresponding to a parking function.

#### Prime Parking Functions

In [50]:
print("Seq. of Sizes of Prime Parking Functions:")

result = []
for n in range(1, 6):
    pfset = basic_parking_functions(n)
    prime_set = are_prime(pfset, n)
    result.append(len(prime_set))
print(result)

Seq. of Sizes of Prime Parking Functions:
[1, 1, 4, 27, 256]


In [51]:
for n in range(3, 5):
    print("Prime Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    prime_set = are_prime(pfset, n)
    result = run_homomesic_on_PFs(prime_set)
    if result is not None:
        print(result)
    print("\n")

Prime Parking Functions: n = 3
293 195
293 942
293 1903
298 195
301 195
304 195
304 942
304 1903


Prime Parking Functions: n = 4
293 942
293 1903
304 942
304 1903




In [52]:
for n in range(3, 6):
    print("\nPrime Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    prime_set = are_prime(pfset, n)
    sagePFs = [ParkingFunction(pf) for pf in prime_set]
    print(find_homometricty(sagePFs))


Prime Parking Functions: n = 3

Bijection: 293, Statistic: 135, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 1.67
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 1.0

Bijection: 293, Statistic: 136, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 0.33
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 165, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 4.0
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 3.0

Bijection: 293, Statistic: 188, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 2.0
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 3.0

Bijection: 293, Statistic: 194, Num of Diff. Orbit Length: 2
Orbit Length: 3
  Orbit: [[1, 2, 1], [2, 1, 1], [1, 1, 2]], Average: 0.33
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 195, Nu

For prime parking functions, we have observed that our same map as before exhibits homomesy with this set and anotehr statistic in addtion to the classical case (but not the picky one)-- Stat942, the number of critical left to right maxima of the parking functions.

#### Fubini Rankings

In [53]:
print("Seq. of Sizes of Fubini Rankings:")

result = []
for n in range(1, 7):
    pfset = basic_parking_functions(n)
    fubini_set = are_fubini_rankings(pfset)
    result.append(len(fubini_set))
print(result)

Seq. of Sizes of Fubini Rankings:
[1, 3, 13, 75, 541, 4683]


In [54]:
for n in range(2, 5):
    print("Fubini Rankings: n = " + str(n))
    pfset = basic_parking_functions(n)
    fubini_set = are_fubini_rankings(pfset)
    result = run_homomesic_on_PFs(fubini_set)
    if result is not None:
        print(result)
    print("\n")

Fubini Rankings: n = 2
293 195
293 1903
298 195
299 195
300 195
301 195
304 195
304 1903


Fubini Rankings: n = 3
293 1903
304 1903


Fubini Rankings: n = 4
293 1903
304 1903




In [55]:
for n in range(4, 6):
    print("\nFubini Rankings: n = " + str(n))
    pfset = basic_parking_functions(n)
    fubini_set = are_fubini_rankings(pfset)
    sagePFs = [ParkingFunction(pf) for pf in fubini_set]
    print(find_homometricty(sagePFs))


Fubini Rankings: n = 4

Bijection: 293, Statistic: 1903, Num of Diff. Orbit Length: 3
Orbit Length: 4
  Orbit: [[3, 2, 1, 3], [2, 1, 3, 3], [1, 3, 3, 2], [3, 3, 2, 1]], Average: 1.0
  Orbit: [[2, 3, 1, 4], [3, 1, 4, 2], [1, 4, 2, 3], [4, 2, 3, 1]], Average: 1.0
  Orbit: [[2, 3, 3, 1], [3, 3, 1, 2], [3, 1, 2, 3], [1, 2, 3, 3]], Average: 1.0
  Orbit: [[3, 1, 1, 3], [1, 1, 3, 3], [1, 3, 3, 1], [3, 3, 1, 1]], Average: 1.0
  Orbit: [[1, 2, 3, 4], [2, 3, 4, 1], [3, 4, 1, 2], [4, 1, 2, 3]], Average: 1.0
  Orbit: [[1, 2, 4, 3], [2, 4, 3, 1], [4, 3, 1, 2], [3, 1, 2, 4]], Average: 1.0
  Orbit: [[1, 3, 2, 3], [3, 2, 3, 1], [2, 3, 1, 3], [3, 1, 3, 2]], Average: 1.0
  Orbit: [[1, 1, 4, 1], [1, 4, 1, 1], [4, 1, 1, 1], [1, 1, 1, 4]], Average: 1.0
  Orbit: [[4, 3, 2, 1], [3, 2, 1, 4], [2, 1, 4, 3], [1, 4, 3, 2]], Average: 1.0
  Orbit: [[4, 1, 3, 2], [1, 3, 2, 4], [3, 2, 4, 1], [2, 4, 1, 3]], Average: 1.0
  Orbit: [[1, 4, 3, 1], [4, 3, 1, 1], [3, 1, 1, 4], [1, 1, 4, 3]], Average: 1.0
  Orbit: [[4, 2, 

The fubini rankings themsleves only have the same case of homomesy as the classical case; nothing to noteworthy.

#### Primary Parking Functions

In [56]:
print("Seq. of Sizes of Primary Rankings:")

result = []
for n in range(1, 8):
    pfset = basic_parking_functions(n)
    primary_set = is_primary(pfset)
    result.append(len(primary_set))
print(result)

Seq. of Sizes of Primary Rankings:
[1, 2, 6, 24, 120, 720, 5040]


In [57]:
for n in range(2, 6):
    print("Primary Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    primary_set = is_primary(pfset)
    result = run_homomesic_on_PFs(primary_set)
    if result is not None:
        print(result)
    print("\n")

Primary Parking Functions: n = 2
293 195
293 1903
298 195
298 1937
299 195
299 1937
300 195
300 1937
301 195
301 1937
304 195
304 1903


Primary Parking Functions: n = 3
293 1903
298 195
299 195
299 1937
300 195
300 1937
301 195
304 1903


Primary Parking Functions: n = 4
293 1903
304 1903


Primary Parking Functions: n = 5
293 1903
304 1903




In [58]:
for n in range(3, 6):
    print("\nPrimary Parking Functions: n = " + str(n))
    pfset = basic_parking_functions(n)
    primary_set = is_primary(pfset)
    sagePFs = [ParkingFunction(pf) for pf in primary_set]
    print(find_homometricty(sagePFs))


Primary Parking Functions: n = 3

Bijection: 293, Statistic: 195, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 2, 1]], Average: 0.0
  Orbit: [[1, 1, 3]], Average: 0.0
  Orbit: [[1, 2, 3]], Average: 0.0
  Orbit: [[1, 1, 2]], Average: 0.0
  Orbit: [[1, 2, 2]], Average: 0.0
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 1937, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 2, 1]], Average: 3.0
  Orbit: [[1, 1, 3]], Average: 3.0
  Orbit: [[1, 2, 3]], Average: 3.0
  Orbit: [[1, 1, 2]], Average: 3.0
  Orbit: [[1, 2, 2]], Average: 3.0
  Orbit: [[1, 1, 1]], Average: 3.0

Bijection: 298, Statistic: 195, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 2, 1]], Average: 0.0
  Orbit: [[1, 1, 3]], Average: 0.0
  Orbit: [[1, 1, 2]], Average: 0.0
  Orbit: [[1, 1, 1]], Average: 0.0
Orbit Length: 2
  Orbit: [[1, 2, 3], [1, 2, 2]], Average: 0.0

Bijection: 298, Statistic: 1937, Num of Diff. Orbit Length: 2
Orbit Length: 1
  Orbit: [[1, 2, 1]], Average:

Primary parking functions, which we need to investgate further, also only show the classical parking function case for homomesy; however, when find_homometricity is run we seem to have cases for homomesy so we need to investigate further.

#### Friendship Parking Functions

In [59]:
print("Seq. of Sizes of Friendship Parking Functions: TK")

Seq. of Sizes of Friendship Parking Functions: TK


In [60]:
for n in range(3, 5):
    print("Friendship Parking Functions: n = " + str(n))
    pfset = generate_friendship_pfs(n)
    for graph, pfs in pfset.items():
        if graph:  # Skip graphs with no edges
            friendship_pf_set = []
            friendship_pf_set.append(pfs)
            print(f"Graph: {graph}; Set-Size: {len(pfs)}")
            result = run_homomesic_on_PFs(friendship_pf_set[0])
            if result is not None:
                print(result)
            print("\n")

Friendship Parking Functions: n = 3
Graph: ((1, 2),); Set-Size: 10
293 1903
304 1903


Graph: ((1, 3),); Set-Size: 10
293 1903


304 1903


Graph: ((2, 3),); Set-Size: 9
293 1903
304 1903


Graph: ((1, 2), (1, 3)); Set-Size: 11
293 1903
304 1903


Graph: ((1, 2), (2, 3)); Set-Size: 11
293 1903
304 1903


Graph: ((1, 3), (2, 3)); Set-Size: 11
293 1903
304 1903


Graph: ((1, 2), (1, 3), (2, 3)); Set-Size: 12
293 1903
304 1903


Friendship Parking Functions: n = 4
Graph: ((1, 2),); Set-Size: 81
293 1903
304 1903


Graph: ((1, 3),); Set-Size: 84
293 1903
304 1903


Graph: ((1, 4),); Set-Size: 86
293 1903
304 1903


Graph: ((2, 3),); Set-Size: 81
293 1903
304 1903


Graph: ((2, 4),); Set-Size: 82
293 1903
304 1903


Graph: ((3, 4),); Set-Size: 81
293 1903
304 1903


Graph: ((1, 2), (1, 3)); Set-Size: 84
293 1903
304 1903


Graph: ((1, 2), (1, 4)); Set-Size: 85
293 1903
304 1903


Graph: ((1, 2), (2, 3)); Set-Size: 87
293 1903
304 1903


Graph: ((1, 2), (2, 4)); Set-Size: 88
293 1903
304 1903


Graph: ((1, 2), (3, 4)); Set-Size: 81
293 1903
304 1903


Graph: ((1, 3), (1, 4)); Set-Size: 88
293 1903
304 1903


Graph: ((

In [61]:
for n in range(3, 6):
    print("\nFriendship Parking Functions: n = " + str(n))
    pfset = generate_friendship_pfs(n)
    sagePFs = [ParkingFunction(pfs[0]) for graph, pfs in pfset.items()]
    print(find_homometricty(sagePFs))


Friendship Parking Functions: n = 3

Bijection: 293, Statistic: 135, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 1.0

Bijection: 293, Statistic: 136, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 165, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 3.0

Bijection: 293, Statistic: 188, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 3.0

Bijection: 293, Statistic: 194, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 195, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 540, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 942, Num of Diff. Orbit Length: 1
Orbit Length: 1
  Orbit: [[1, 1, 1]], Average: 0.0

Bijection: 293, Statistic: 943, Num of Diff. Orbit Length:

We are unsure about the current state of the code for friendship parking functions; it mst be reviewed for the complete case.

#### Unit Fubini Rankings

In [62]:
print("Seq. of Sizes of Unit Fubini Rankings:")

result = []
for n in range(1, 8):
    unit_fubini_set = generate_unit_fubini(n)
    result.append(len(unit_fubini_set))
print(result)

Seq. of Sizes of Unit Fubini Rankings:
[1, 3, 11, 55, 343, 2561, 22329]


In [63]:
for n in range(2, 5):
    print("Unit Fubini Rankings: n = " + str(n))
    unit_fubini_set = generate_unit_fubini(n)
    result = run_homomesic_on_PFs(unit_fubini_set)
    if result is not None:
        print(result)
    print("\n")

Unit Fubini Rankings: n = 2
293 195
293 1903
298 195
299 195
300 195
301 195
304 195
304 1903


Unit Fubini Rankings: n = 3
293 1903
304 1903


Unit Fubini Rankings: n = 4
293 1903
304 1903




In [64]:
for n in range(3, 6):
    print("Unit Fubini Rankings: n = " + str(n))
    unit_fubini_set = generate_unit_fubini(n)
    sagePFs = [ParkingFunction(pfs) for pfs in unit_fubini_set]
    print(find_homometricty(sagePFs))
    
print(generate_unit_fubini(4))

Unit Fubini Rankings: n = 3

Bijection: 303, Statistic: 135, Num of Diff. Orbit Length: 2
Orbit Length: 2
  Orbit: [[1, 3, 2], [3, 1, 2]], Average: 3.0
  Orbit: [[1, 2, 3], [3, 2, 1]], Average: 3.0
  Orbit: [[2, 1, 3], [2, 3, 1]], Average: 3.0
Orbit Length: 1
  Orbit: [[1, 1, 3]], Average: 2.0
  Orbit: [[1, 3, 1]], Average: 2.0
  Orbit: [[1, 2, 2]], Average: 2.0
  Orbit: [[2, 1, 2]], Average: 2.0
  Orbit: [[3, 1, 1]], Average: 2.0

Bijection: 303, Statistic: 165, Num of Diff. Orbit Length: 2
Orbit Length: 2
  Orbit: [[1, 3, 2], [3, 1, 2]], Average: 6.0
  Orbit: [[1, 2, 3], [3, 2, 1]], Average: 6.0
  Orbit: [[2, 1, 3], [2, 3, 1]], Average: 6.0
Orbit Length: 1
  Orbit: [[1, 1, 3]], Average: 5.0
  Orbit: [[1, 3, 1]], Average: 5.0
  Orbit: [[1, 2, 2]], Average: 5.0
  Orbit: [[2, 1, 2]], Average: 5.0
  Orbit: [[3, 1, 1]], Average: 5.0

Bijection: 303, Statistic: 188, Num of Diff. Orbit Length: 2
Orbit Length: 2
  Orbit: [[1, 3, 2], [3, 1, 2]], Average: 0.0
  Orbit: [[1, 2, 3], [3, 2, 1]], A

Unit Fubini Rankings unfortunately also don't show any other cases of homomesy or homometry other then the one for classical parking functions.   

### Sage function for generating homometery (and therefore homomesy) disproofs for all triples (set, map, stat)

In [29]:
def generate_disproof(map_id, stat_id, map_name, stat_name, pf_len_n, pf1, pf2, orbit_pf1, orbit_pf2, numerator_pf1, denominator_pf1, result_pf1, numerator_pf2, denominator_pf2, result_f2):
    latex_disproof = r"""\newcommand{\disproof}[9]{
        \noindent \subsection{Map#1, Stat#2:}
        Let $B$ be the #3 map, and $\mathcal{X}$ be the #4. Consider the following parking functions from $PF_#5$: $f_1 = #6$ and $f_2 = #7$ with orbits $\mathcal{O}(f_1) = \{#8\}$ and $\mathcal{O}(f_2) = \{#9\}$.
        Then
        }

        \newcommand{\showsteps}[7]{
        $$ \frac{1}{{\left|\mathcal{O}(f_1)\right|}}\sum_{g \in \mathcal{O}(f_1)}\mathcal{X}(g) = \frac{#1}{#2} = #3, \quad \frac{1}{{\left|\mathcal{O}(f_2)\right|}}\sum_{g \in \mathcal{O}(f_2)}\mathcal{X}(g) = \frac{#4}{#5} = #6.$$
        As $#3 \neq #6$, we have shown $(PF_#7, B, \mathcal{X})$ is not homomesic.
        \vspace{5pt}
        }
        """
    
    disproof_filled = r"\disproof{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}".format(
        map_id, stat_id, map_name, stat_name, pf_len_n, pf1, pf2, orbit_pf1, orbit_pf2
    )
    
    showsteps_filled = r"\showsteps{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}{{{}}}".format(
        numerator_pf1, denominator_pf1, round_result(result_pf1), numerator_pf2, denominator_pf2, round_result(result_f2), pf_len_n
    )
    
    full_latex = disproof_filled + "\n" + showsteps_filled
    return full_latex

def round_result(number):
    if isinstance(number, int) or number.is_integer():
        return number
    else:
        return round(number, 2)

# Real Example
map_number = 293
stat_number = 2
map_name = "roatate front-to-back"
x_name = "length of the initial strictly increasing segment of a parking function"
pf_len = 3
f1 = (1, 1, 1)
f2 = (1, 3, 1)
orbit_f1 = "(1, 1, 1)"
orbit_f2 = "(3, 1, 1), (1, 1, 3), (1, 3, 1)"
numerator_f1 = 1
denominator_f1 = 1
result_f1 = 1
numerator_f2 = "(1+2+1)"
denominator_f2 = 3
result_f2 = 1.33

# Generate the LaTeX code
latex_code = generate_disproof(map_number, stat_number, map_name, x_name, pf_len, f1, f2, orbit_f1, orbit_f2, numerator_f1, denominator_f1, result_f1, numerator_f2, denominator_f2, result_f2)
print(latex_code)

\disproof{293}{2}{roatate front-to-back}{length of the initial strictly increasing segment of a parking function}{3}{(1, 1, 1)}{(1, 3, 1)}{(1, 1, 1)}{(3, 1, 1), (1, 1, 3), (1, 3, 1)}
\showsteps{1}{1}{1}{(1+2+1)}{3}{1.33}{3}


### FindStat's total displacement stat

The total displacement of a parking function.


The total displacement of a parking function $ p $ is defined by
$$
\operatorname{disp}(p) := \sum_{i=1}^{n} d_i,
$$
where the displacement vector $ d := (d_1, d_2, \ldots, d_n) $ and $ d_i := \pi^{-1}(i) - p_i $ for all $ i \in [n] $, 
such that each $ d_i $ is the positive difference between the actual spot a car parks and its preferred spot.

In [66]:
# Ref: [1] [[arXiv:2310.06560]]
def statistic(p):
    """
    Calculates the total displacement of a parking function.

    Parameters:
    p (ParkingFunction): A parking function, represented as a list of integers.

    Returns:
    int: The total displacement of the parking function.
    """
    n = len(p)
    lot = [None] * n
    total_disp = 0

    for i in range(n):
        pref = p[i] - 1 
        curr = pref
        
        while curr < n and lot[curr] is not None:
            curr += 1
        
        if curr < n:
            lot[curr] = i
            total_disp += curr - pref

    return total_disp

    # Added to Stat188 in FindStat database

### Check bijection(s) on 'Picky' and Prime parking functions

In [67]:
def check_bijection(n, pf_name, subset_func, mapNum):
    print(f"{pf_name.capitalize()} Parking Functions | n = {n}")
    pfset = basic_parking_functions(n)
    subset = subset_func(pfset, n) if pf_name == 'prime' else subset_func(pfset)
    
    bijection = findmap(mapNum)

    # Apply the bijection to each parking function in the specific set
    output_set = [bijection(pf) for pf in subset]

    # Convert both sets to sets of tuples for comparison
    specific_set_as_tuples = {tuple(p) for p in subset}
    output_set_as_tuples = {tuple(p) for p in subset}

    # Find elements in output_set that are not in specific_set
    difference_set = output_set_as_tuples - specific_set_as_tuples

    # Print the elements that are in output_set but not in specific_set
    if difference_set:
        print(f"Elements in output_set but not in {pf_name} set for n = {n}:")
        for pf in difference_set:
            print(pf)
    else:
        print(f"All elements in output_set are in {pf_name} set for n = {n}\n")

# Test roatate front-to-back map on Prime and Picky parking functions
for n in range(2, 6):
    check_bijection(n, 'prime', are_prime, 293)

for n in range(2, 6):
    check_bijection(n, 'picky', are_picky, 293)

Prime Parking Functions | n = 2
All elements in output_set are in prime set for n = 2

Prime Parking Functions | n = 3
All elements in output_set are in prime set for n = 3

Prime Parking Functions | n = 4
All elements in output_set are in prime set for n = 4

Prime Parking Functions | n = 5
All elements in output_set are in prime set for n = 5

Picky Parking Functions | n = 2
All elements in output_set are in picky set for n = 2

Picky Parking Functions | n = 3
All elements in output_set are in picky set for n = 3

Picky Parking Functions | n = 4
All elements in output_set are in picky set for n = 4

Picky Parking Functions | n = 5
All elements in output_set are in picky set for n = 5



### Redefine bijections

In [78]:
def orbit_redefinition(super_set, sub_set, bij_map):
    result = []
    sub_set = (tuple(pf) for pf in sub_set)  # Convert each sub_set element to a tuple
    orbits = orbit_decomposition([tuple(pf) for pf in super_set], bij_map)  # Convert each super_set element to a tuple

    for orbit in orbits:
        new_orbit = []
        for pf in orbit:
            if pf in sub_set:
                new_orbit.append(pf)
        if new_orbit:
            result.append(new_orbit)

    return result

def subset_homomesy(pfs, subset):
    result = []
    for bijection in [293, 298, 299, 300, 301, 303, 304, 320, 52]:
        bij = findmap(bijection)
        orbits = orbit_redefinition(pfs, subset, bij)
        for statistic in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]:
            stat = findstat(statistic)
            grouped_averages = defaultdict(list)
            
            for orbit in orbits:
                orb_stat_avg = round(sum([stat(p) for p in orbit]) / float(len(orbit)), 2)
                orbit_length = len(orbit)
                grouped_averages[orbit_length].append((orbit, orb_stat_avg))
            
            num_different_orbit_lengths = len(grouped_averages)

            # Flag to check if all avg_values in all orbit_length buckets are the same
            all_same = True
            for orbit_length, stats in grouped_averages.items():
                avg_values = [stat[1] for stat in stats]
                if len(set(avg_values)) != 1:
                    all_same = False
                    break

            if all_same:
                print(f"\nBijection: {bijection}, Statistic: {statistic}, Num of Diff. Orbit Length: {num_different_orbit_lengths}")
                for orbit_length, stats in grouped_averages.items():
                    print(f"Orbit Length: {orbit_length}")
                    for orbit, avg in stats:
                        print(f"  Orbit: {orbit}, Average: {avg}")

In [81]:
for n in range(3, 6):
    pfset = basic_parking_functions(n)
    subset = are_prime(pfset, n)

    sage_pfset = [ParkingFunction(pf) for pf in pfset]
    sage_subset = [ParkingFunction(pf) for pf in subset]
    
    subset_homomesy(sage_pfset, sage_subset)

TypeError: can only concatenate tuple (not "list") to tuple

### 'Picky' Parking Functions are Cayley Permutations!

In [69]:
# from[[https://11011110.github.io/blog/2013/03/13/cayley-permutations.html]]
def CayleyPermutations(n):
    """Generate sequence of Cayley permutations of length n"""
    if n < 2:
        yield [1]*n
        return
    for P in CayleyPermutations(n-1):
        m = max(P)
        i = n-1
        P = P + [m+1]
        pastMax = False
        yield P
        while i > 0:
            if not pastMax:
                P[i] = m
                yield P
                if P[i-1] == m:
                    pastMax = True
            P[i] = P[i-1]
            P[i-1] = m+1
            i -= 1
            yield P

for n in range(1, 8):
    pfset = basic_parking_functions(n)
    picky_set = are_picky(pfset)
    cayley_permutations = CayleyPermutations(n)

    # Convert lists of lists to sets of tuples
    picky_set_as_tuples = {tuple(p) for p in picky_set}
    cayley_permutations_as_tuples = {tuple(p) for p in cayley_permutations}

    # Compare the sets
    if picky_set_as_tuples == cayley_permutations_as_tuples:
        print("True @ n = " + str(n))
    else:
        print("False @ n = " + str(n))

True @ n = 1
True @ n = 2
True @ n = 3
True @ n = 4
True @ n = 5
True @ n = 6
True @ n = 7


### Random tests area

In [3]:
from sage.combinat.parking_functions import *
from sage.combinat.cyclic_sieving_phenomenon import *
from sage.dynamics.finite_dynamical_system import *
from sage.databases.findstat import *
findstat()._allow_execution = True

# Define your k-Naples Parking Functions generation
def k_naples_l_interval_m_cars_n_spots_parking_functions(k, l, m, n):
    # Define the basic variables descriptively 
    max_back_steps = k # k-naples
    max_forward_steps = l # l-interval 
    total_cars = m # m-cars
    total_spots = n # n-spots
    
    street = [0] * (total_spots + 1) # Initialize a street with all spots empty
    preferences, parking_functions = [], [] # Initialize lists for preferences and valid parking functions

    def helper(cars_parked): 
        if cars_parked == total_cars: # Base case: all cars are parked
            parking_functions.append(preferences[:]) # Add a copy of the current preference list to parking functions
            return 1
            
        count = 0
        for preference_value in range(total_spots + 1):  # Iterate through all possible preference values
            spot = preference_value
            while street[spot] != 0 and preference_value - spot < max_back_steps and spot > 0:
                spot -= 1  # Move backward if the spot is occupied and within max_back_steps
            while street[spot] != 0 and spot - preference_value <= max_forward_steps:
                spot += 1  # Move forward if the spot is occupied and within max_forward_steps
                if spot > total_spots:  # Break if moving beyond the last spot
                    break
            if spot == total_spots or spot - preference_value > max_forward_steps:
                continue  # Skip if the car can't be parked within the constraints

            street[spot] = 1  # Mark the spot as occupied
            preferences.append(preference_value + 1)  # Add the preference value to the list
            count += helper(cars_parked + 1)  # Recur to park the next car
            street[spot] = 0  # Backtrack: unmark the spot
            preferences.pop()  # Backtrack: remove the preference value
        return count

    helper(0) # Start the recursive parking with 0 cars parked
    return parking_functions # Return all valid parking functions

def k_naples_parking_functions(k, n):
    return k_naples_l_interval_m_cars_n_spots_parking_functions(k, n, n, n)

def unit_interval_parking_functions(n): # k = 1, l = 1, and m = n
    return k_naples_l_interval_m_cars_n_spots_parking_functions(1, 1, n, n)

def run_homomesic_on_PFs(pfs): 
    findstat()._allow_execution = True
    for map in FindStatMaps(domain='Cc0023', codomain='Cc0023'):
        if map.properties_raw().find('bijective') >= 0:
            pfset = [ParkingFunction(pf) for pf in pfs]  # Convert each list to a ParkingFunction object
            F = DiscreteDynamicalSystem(pfset, map)
            for k in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]: # all PF stats on FindState minus the ones without codes
                stat = findstat(k)
                if F.is_homomesic(stat):
                    print(map.id(), stat.id())

def find_homometricty(pfs):
    result = []
    for bijection in [293, 298, 299, 300, 301, 303, 304, 320, 52]:
        bij = findmap(bijection)
        for statistic in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]:
            orbits = orbit_decomposition(pfs, bij)
            stat = findstat(statistic)
            grouped_averages = defaultdict(list)
            
            for orbit in orbits:
                orb_stat_avg = round(sum([stat(p) for p in orbit]) / float(len(orbit)), 2)
                orbit_length = len(orbit)
                grouped_averages[orbit_length].append((orbit, orb_stat_avg))
            
            num_different_orbit_lengths = len(grouped_averages)

            # Flag to check if all avg_values in all orbit_length buckets are the same
            all_same = True
            for orbit_length, stats in grouped_averages.items():
                avg_values = [stat[1] for stat in stats]
                if len(set(avg_values)) != 1:
                    all_same = False
                    break

            if all_same:
                print(f"\nBijection: {bijection}, Statistic: {statistic}, Num of Diff. Orbit Length: {num_different_orbit_lengths}")
                for orbit_length, stats in grouped_averages.items():
                    print(f"Orbit Length: {orbit_length}")
                    for orbit, avg in stats:
                        print(f"  Orbit: {orbit}, Average: {avg}")

# Example usage
for n in range(2, 6):
    print("Unit Interval Parking Functions: n =", i, ", k =", k)
    pfs = unit_interval_parking_functions(n)
    psage = [ParkingFunction(p) for p in pfs]
    print(find_homometricty(psage))
    print("\n")

k-Naples Parking Functions: n = 2 , k = 1


ValueError: [2, 2] is not a parking function

In [ ]:
pf = ParkingFunction([6, 1, 5, 2, 2, 1, 5])
print(pf.secondary_dinversion_pairs())
print(pf.to_area_sequence())
print(pf.to_labelling_permutation())

from sage.combinat.words.word import Word
print(Word(pf))
print(Word(pf).standard_permutation())

[(1, 4), (2, 4), (3, 6)]
[0, 1, 1, 2, 0, 1, 1]
[2, 6, 4, 5, 3, 7, 1]
6152215
[7, 1, 5, 3, 4, 2, 6]


In [14]:
for n in range(2, 6): 
    pfset = basic_parking_functions(n)
    primary_set = is_primary(pfset)
    print(primary_set)

[[1, 1], [1, 2]]
[[1, 1, 1], [1, 1, 2], [1, 1, 3], [1, 2, 1], [1, 2, 2], [1, 2, 3]]
[[1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 1, 3], [1, 1, 1, 4], [1, 1, 2, 1], [1, 1, 2, 2], [1, 1, 2, 3], [1, 1, 2, 4], [1, 1, 3, 1], [1, 1, 3, 2], [1, 1, 3, 3], [1, 1, 3, 4], [1, 2, 1, 1], [1, 2, 1, 2], [1, 2, 1, 3], [1, 2, 1, 4], [1, 2, 2, 1], [1, 2, 2, 2], [1, 2, 2, 3], [1, 2, 2, 4], [1, 2, 3, 1], [1, 2, 3, 2], [1, 2, 3, 3], [1, 2, 3, 4]]
[[1, 1, 1, 1, 1], [1, 1, 1, 1, 2], [1, 1, 1, 1, 3], [1, 1, 1, 1, 4], [1, 1, 1, 1, 5], [1, 1, 1, 2, 1], [1, 1, 1, 2, 2], [1, 1, 1, 2, 3], [1, 1, 1, 2, 4], [1, 1, 1, 2, 5], [1, 1, 1, 3, 1], [1, 1, 1, 3, 2], [1, 1, 1, 3, 3], [1, 1, 1, 3, 4], [1, 1, 1, 3, 5], [1, 1, 1, 4, 1], [1, 1, 1, 4, 2], [1, 1, 1, 4, 3], [1, 1, 1, 4, 4], [1, 1, 1, 4, 5], [1, 1, 2, 1, 1], [1, 1, 2, 1, 2], [1, 1, 2, 1, 3], [1, 1, 2, 1, 4], [1, 1, 2, 1, 5], [1, 1, 2, 2, 1], [1, 1, 2, 2, 2], [1, 1, 2, 2, 3], [1, 1, 2, 2, 4], [1, 1, 2, 2, 5], [1, 1, 2, 3, 1], [1, 1, 2, 3, 2], [1, 1, 2, 3, 3], [1, 1, 2, 3, 4], 

TODO!

In [60]:
from sage.combinat.permutation import Permutations
from collections import defaultdict

# Finds homometry triples given a set
def find_homometricty(pfs):
    result = []
    for bijection in [293, 298, 299, 300, 301, 303, 304, 320, 52]:
        bij = findmap(bijection)
        for statistic in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]:
            orbits = orbit_decomposition(pfs, bij)
            stat = findstat(statistic)
            grouped_averages = defaultdict(list)
            
            for orbit in orbits:
                orb_stat_avg = round(sum([stat(p) for p in orbit]) / float(len(orbit)), 2)
                orbit_length = len(orbit)
                grouped_averages[orbit_length].append((orbit, orb_stat_avg))
            
            num_different_orbit_lengths = len(grouped_averages)

            # Flag to check if all avg_values in all orbit_length buckets are the same
            all_same = True
            for orbit_length, stats in grouped_averages.items():
                avg_values = [stat[1] for stat in stats]
                if len(set(avg_values)) != 1:
                    all_same = False
                    break

            if all_same:
                print(f"\nBijection: {bijection}, Statistic: {statistic}, Num of Diff. Orbit Length: {num_different_orbit_lengths}")
                for orbit_length, stats in grouped_averages.items():
                    print(f"Orbit Length: {orbit_length}")
                    for orbit, avg in stats:
                        print(f"  Orbit: {orbit}, Average: {avg}")

def are_inversion_seqs(perm_set):
    result = []
    for perm in perm_set:
        inv_seq = []
        for i in range(len(perm)):
            count = 0
            for j in range(i):
                if perm[j] > perm[i]:
                    count += 1
            inv_seq.append(count)
        result.append(inv_seq)
    return result

def are_inversion_tables(perm_set):
    result = []
    for perm in perm_set:
        inv_table = []
        n = len(perm)
        for i in range(1, n + 1):
            count = 0
            try:
                pos = perm.index(i - 1)
            except ValueError:
                pos = 1
            for j in range(pos):
                if perm[j] > (i - 1):
                    count += 1
            inv_table.append(count)
        result.append(inv_table)
    return result

# Example case
perm_set = [[5, 2, 0, 4, 3, 1]]
print(are_inversion_tables(perm_set))
print(are_inversion_seqs(perm_set))
print()

def generate_catalan_words(n):
    if n < 1:
        return []

    def backtrack(word, length):
        if length == n:
            catalan_words.append(word)
            return
        
        max_value = word[-1] + 1
        for i in range(max_value + 1):
            backtrack(word + [i], length + 1)

    catalan_words = []
    backtrack([0], 1)  # Starting with w_1 = 0
    return catalan_words

# Example usage
n = 3
catalan_words = generate_catalan_words(n)
for word in catalan_words:
    print(word)
print()

seq_of_cardinalitesSEQ, seq_of_cardinalitesTABLE = [], []
seq_of_cardinalitesCW = []
for n in range(1, 8):
    print(f"Permutations @ n = {n}")
    perm_set = Permutations(n).list()
    inv_seq_set = are_inversion_seqs(perm_set)
    inv_table_set = are_inversion_tables(perm_set)
    catalan_words = generate_catalan_words(n)
    
    cw_add_1_set = []
    for word in catalan_words:
        new_word = [num + 1 for num in word]
        cw_add_1_set.append(new_word)

    seq_of_cardinalitesCW.append(len(catalan_words))
    seq_of_cardinalitesSEQ.append(len(inv_seq_set))
    seq_of_cardinalitesTABLE.append(len(inv_table_set))
    if set(tuple(x) for x in is_primary(basic_parking_functions(n))) == set(tuple(x) for x in inv_seq_set):
        print(f"Same Set | Seqs")
    if set(tuple(x) for x in is_primary(basic_parking_functions(n))) == set(tuple(x) for x in inv_table_set):
        print(f"Same Set | Tables")
    if set(tuple(x) for x in is_primary(basic_parking_functions(n))) == set(tuple(x) for x in cw_add_1_set):
        print(f"Same Set | CW")
print(seq_of_cardinalitesSEQ)
print(seq_of_cardinalitesTABLE)
print(seq_of_cardinalitesCW)

# Example usage
for n in range(4, 6):
    perm_set = Permutations(n).list()
    inv_seq_set = are_inversion_seqs(perm_set)
    print(subset_homomesy(perm_set, inv_seq_set))


[[2, 4, 1, 2, 1, 0]]
[[0, 1, 2, 1, 2, 4]]

[0, 0, 0]
[0, 0, 1]
[0, 1, 0]
[0, 1, 1]
[0, 1, 2]

Permutations @ n = 1
Same Set | Tables
Same Set | CW
Permutations @ n = 2
Same Set | CW
Permutations @ n = 3
Permutations @ n = 4
Permutations @ n = 5
Permutations @ n = 6
Permutations @ n = 7
[1, 2, 6, 24, 120, 720, 5040]
[1, 2, 6, 24, 120, 720, 5040]
[1, 2, 5, 14, 42, 132, 429]


TypeError: unhashable type: 'list'

In [41]:
def generate_stirling_permutations(n):
    def backtrack(current_perm, used, n, result):
        if len(current_perm) == 2 * n:
            result.append(current_perm.copy())
            return
        
        for num in range(1, n + 1):
            if used[num] < 2:
                current_perm.append(num)
                used[num] += 1
                backtrack(current_perm, used, n, result)
                used[num] -= 1
                current_perm.pop()

    result = []
    used = {i: 0 for i in range(1, n + 1)}
    backtrack([], used, n, result)
    return result

def is_valid(permutation):
    positions = {}
    for idx, num in enumerate(permutation):
        if num not in positions:
            positions[num] = idx
        else:
            if any(permutation[i] < num for i in range(positions[num] + 1, idx)):
                return False
            positions[num] = idx
    return True


# tests all bijections and statistics for homomesy | Sage Compliant | Cite Elder's et al. Papers
def run_homomesic_on_PFs(pfs): 
    findstat()._allow_execution = True
    for function in FindStatMaps(domain='Cc0023', codomain='Cc0023'):
        if function.properties_raw().find('bijective') >= 0:
            pfset = pfs
            F = DiscreteDynamicalSystem(pfset, function)
            for stat in pf_stats_wo_errors: # all PF stats on FindState minus the ones without codes
                if F.is_homomesic(stat): # prints map and stat that is homomesic under the set of given Parking Functions
                    print(function.id(), stat.id())

# Finds homometry triples given a set
def find_homometricty(pfs):
    result = []
    for bijection in [293, 298, 299, 300, 301, 303, 304, 320, 52]:
        bij = findmap(bijection)
        for statistic in [135, 136, 165, 188, 194, 195, 540, 942, 943, 1209, 1903, 1904, 1905, 1935, 1937]:
            orbits = orbit_decomposition(pfs, bij)
            stat = findstat(statistic)
            grouped_averages = defaultdict(list)
            
            for orbit in orbits:
                orb_stat_avg = round(sum([stat(p) for p in orbit]) / float(len(orbit)), 2)
                orbit_length = len(orbit)
                grouped_averages[orbit_length].append((orbit, orb_stat_avg))
            
            num_different_orbit_lengths = len(grouped_averages)

            # Flag to check if all avg_values in all orbit_length buckets are the same
            all_same = True
            for orbit_length, stats in grouped_averages.items():
                avg_values = [stat[1] for stat in stats]
                if len(set(avg_values)) != 1:
                    all_same = False
                    break

            if all_same:
                print(f"\nBijection: {bijection}, Statistic: {statistic}, Num of Diff. Orbit Length: {num_different_orbit_lengths}")
                # for orbit_length, stats in grouped_averages.items():
                    # print(f"Orbit Length: {orbit_length}")
                    # for orbit, avg in stats:
                        # print(f"  Orbit: {orbit}, Average: {avg}")

for n in range(5, 6):
    perm_set = generate_stirling_permutations(n)
    result = [] 
    for perm in perm_set:
        if is_valid(perm):
            result.append(ParkingFunction(perm))
    print(f"Length: {len(result)}; Result: {result}")
    print()
    print(find_homometricty(result))
    print()


Length: 945; Result: [[1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [1, 1, 2, 2, 3, 3, 4, 5, 5, 4], [1, 1, 2, 2, 3, 3, 5, 5, 4, 4], [1, 1, 2, 2, 3, 4, 4, 3, 5, 5], [1, 1, 2, 2, 3, 4, 4, 5, 5, 3], [1, 1, 2, 2, 3, 4, 5, 5, 4, 3], [1, 1, 2, 2, 3, 5, 5, 3, 4, 4], [1, 1, 2, 2, 3, 5, 5, 4, 4, 3], [1, 1, 2, 2, 4, 4, 3, 3, 5, 5], [1, 1, 2, 2, 4, 4, 3, 5, 5, 3], [1, 1, 2, 2, 4, 4, 5, 5, 3, 3], [1, 1, 2, 2, 4, 5, 5, 4, 3, 3], [1, 1, 2, 2, 5, 5, 3, 3, 4, 4], [1, 1, 2, 2, 5, 5, 3, 4, 4, 3], [1, 1, 2, 2, 5, 5, 4, 4, 3, 3], [1, 1, 2, 3, 3, 2, 4, 4, 5, 5], [1, 1, 2, 3, 3, 2, 4, 5, 5, 4], [1, 1, 2, 3, 3, 2, 5, 5, 4, 4], [1, 1, 2, 3, 3, 4, 4, 2, 5, 5], [1, 1, 2, 3, 3, 4, 4, 5, 5, 2], [1, 1, 2, 3, 3, 4, 5, 5, 4, 2], [1, 1, 2, 3, 3, 5, 5, 2, 4, 4], [1, 1, 2, 3, 3, 5, 5, 4, 4, 2], [1, 1, 2, 3, 4, 4, 3, 2, 5, 5], [1, 1, 2, 3, 4, 4, 3, 5, 5, 2], [1, 1, 2, 3, 4, 4, 5, 5, 3, 2], [1, 1, 2, 3, 4, 5, 5, 4, 3, 2], [1, 1, 2, 3, 5, 5, 3, 2, 4, 4], [1, 1, 2, 3, 5, 5, 3, 4, 4, 2], [1, 1, 2, 3, 5, 5, 4, 4, 3, 2], [1, 1, 2, 4, 4, 2,

In [58]:
def generate_reverse_primes(n):
    # Generate permutations of length n-1
    perm_set = Permutations(n - 1)
    reverse_primes = []

    for perm in perm_set:
        # Convert permutation to a list
        perm_list = list(perm)
        for i in range(n):
            # Insert 1 at every position in the permutation
            new_perm = perm_list[:i] + [1] + perm_list[i:]
            # Check if the new permutation is a valid parking function
            if new_perm in ParkingFunctions(n):
                reverse_primes.append(new_perm)
                
    return reverse_primes

for n in range(4, 6):
    rp_set = generate_reverse_primes(n)
    rp_pf_set = [ParkingFunction(rp) for rp in rp_set]
    print(f"Length: {len(rp_pf_set)}; Result: {rp_pf_set}")
    print()
    print(find_homometricty(rp_pf_set))
    print()


Length: 24; Result: [[1, 1, 2, 3], [1, 1, 2, 3], [1, 2, 1, 3], [1, 2, 3, 1], [1, 1, 3, 2], [1, 1, 3, 2], [1, 3, 1, 2], [1, 3, 2, 1], [1, 2, 1, 3], [2, 1, 1, 3], [2, 1, 1, 3], [2, 1, 3, 1], [1, 2, 3, 1], [2, 1, 3, 1], [2, 3, 1, 1], [2, 3, 1, 1], [1, 3, 1, 2], [3, 1, 1, 2], [3, 1, 1, 2], [3, 1, 2, 1], [1, 3, 2, 1], [3, 1, 2, 1], [3, 2, 1, 1], [3, 2, 1, 1]]


Bijection: 293, Statistic: 165, Num of Diff. Orbit Length: 1
Orbit Length: 4
  Orbit: [[1, 3, 2, 1], [3, 2, 1, 1], [2, 1, 1, 3], [1, 1, 3, 2]], Average: 7.0
  Orbit: [[1, 1, 2, 3], [1, 2, 3, 1], [2, 3, 1, 1], [3, 1, 1, 2]], Average: 7.0
  Orbit: [[2, 1, 3, 1], [1, 3, 1, 2], [3, 1, 2, 1], [1, 2, 1, 3]], Average: 7.0

Bijection: 293, Statistic: 188, Num of Diff. Orbit Length: 1
Orbit Length: 4
  Orbit: [[1, 3, 2, 1], [3, 2, 1, 1], [2, 1, 1, 3], [1, 1, 3, 2]], Average: 3.0
  Orbit: [[1, 1, 2, 3], [1, 2, 3, 1], [2, 3, 1, 1], [3, 1, 1, 2]], Average: 3.0
  Orbit: [[2, 1, 3, 1], [1, 3, 1, 2], [3, 1, 2, 1], [1, 2, 1, 3]], Average: 3.0

Bijec

### More testing: sweeping FindStat

In [50]:
import itertools

def run_homomesic_on_words(words):
    findstat()._allow_execution = True # To run all the code from Findstat
    for idx in [104, 105, 135, 158, 268, 272, 278, 280, 296]: # Cc0001 is the Permutation Collection
        func = findmap(idx)
        if func.properties_raw().find('bijective') >= 0: # The map is bijective
            F = DiscreteDynamicalSystem(words, func) # Fix n ahead of time
            for k in [288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 326, 347, 348, 389, 390, 391, 392, 393, 
                      518, 519, 529, 543, 626, 627, 628, 629, 630, 631, 682, 691, 753, 792, 826, 827, 847, 875, 
                      876, 877, 878, 885, 921, 922, 982, 983, 1267, 1313, 1355, 1365, 1371, 1372, 1413, 1414, 
                      1415, 1416, 1417, 1419, 1420, 1421, 1423, 1424, 1436, 1437, 1485, 1524, 1721, 1722, 
                      1730, 1838, 1884, 1885, 1915, 1916, 1930]:
                stat = findstat(k)
                if F.is_homomesic(stat):
                    print(func.id(), stat.id())

def generate_binary_words(n):
    return [list(word) for word in itertools.product([0, 1], repeat=n)]

for n in range(4, 6):
    final_set = generate_binary_words(n)
    words_set = [Word(word) for word in final_set]
    print(f"Binary Words | n = {n} | Size = {2^n}")
    run_homomesic_on_words(words_set)
    print()

Binary Words | n = 4 | Size = 16
104 629
105 288
105 289
105 391
105 629
105 827
105 878
135 629
158 629
158 691
158 1524
268 629
272 288
272 289
272 290
272 291
272 292
272 293
272 294
272 295
272 296
272 297
272 326
272 347
272 348
272 389
272 390
272 391
272 392
272 393
272 518
272 519
272 529
272 543
272 626
272 627
272 628
272 629
272 630
272 631
272 682
272 691
272 753
272 792
272 826
272 827
272 847
272 875
272 876
272 877
272 878
272 885
272 921
272 922
272 982
272 983
272 1267
272 1313
272 1355
272 1365
272 1371
272 1372
272 1413
272 1414
272 1415
272 1416
272 1417
272 1419
272 1420
272 1421
272 1423
272 1424
272 1436
272 1437
272 1485
272 1524
272 1721
272 1722
272 1730
272 1838
272 1884
272 1885
272 1915
272 1916
272 1930
278 629
280 288
280 289
280 291
280 292
280 296
280 390
280 391
280 393
280 529
280 629
280 630
280 691
280 827
280 878
280 921
280 1419
280 1424
280 1485
280 1524


AttributeError: 'FiniteWord_list' object has no attribute 'rise_composition'

In [54]:
from math import comb

def run_homomesic_on_dps(words):
    findstat()._allow_execution = True # To run all the code from Findstat
    for idx in [104, 105, 135, 158, 268, 272, 278, 280, 296]: # Cc0001 is the Permutation Collection
        func = findmap(idx)
        if func.properties_raw().find('bijective') >= 0: # The map is bijective
            F = DiscreteDynamicalSystem(words, func) # Fix n ahead of time
            for k in [5, 6, 11, 12, 13, 14, 15, 24, 25, 26, 27, 32, 38, 52, 53, 79, 117, 120, 144, 306, 
                    329, 331, 335, 340, 369, 376, 386, 394, 395, 418, 419, 420, 421, 438, 439, 442, 443, 
                    444, 445, 476, 617, 645, 655, 658, 659, 660, 661, 674, 675, 676, 678, 683, 684, 685, 
                    686, 687, 688, 689, 790, 791, 874, 920, 930, 931, 932, 946, 947, 949, 950, 951, 952, 
                    953, 954, 955, 964, 965, 966, 967, 968, 969, 970, 976, 977, 978, 979, 980, 981, 984, 
                    998, 999, 1000, 1001, 1002, 1003, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 
                    1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1031, 
                    1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1063, 1064, 1065, 1066, 1067, 1068, 1088, 
                    1089, 1104, 1107, 1113, 1125, 1126, 1135, 1137, 1138, 1139, 1140, 1141, 1142, 1159, 1161, 
                    1163, 1164, 1165, 1166, 1167, 1169, 1170, 1172, 1179, 1180, 1181, 1182, 1183, 1184, 1185, 
                    1186, 1187, 1188, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 
                    1201, 1202, 1203, 1204, 1205, 1206, 1210, 1211, 1212, 1213, 1215, 1216, 1217, 1218, 1219, 
                    1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1237, 
                    1238, 1239, 1240, 1241, 1242, 1243, 1244, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1264, 
                    1265, 1266, 1273, 1274, 1275, 1276, 1278, 1289, 1290, 1291, 1292, 1294, 1295, 1296, 1297, 
                    1299, 1314, 1348, 1361, 1418, 1431, 1471, 1473, 1480, 1481, 1483, 1492, 1493, 1498, 1499, 
                    1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1514, 1515, 1523, 1526, 1530, 
                    1531, 1553, 1584, 1594, 1643, 1650, 1669, 1688, 1732, 1733, 1786, 1800, 1808, 1809, 1872, 
                    1873, 1910, 1929, 1932]:
                stat = findstat(k)
                if F.is_homomesic(stat):
                    print(func.id(), stat.id())

def generate_dyck_paths(n):
    def backtrack(path, x, y, start, shut, result):
        if start == n and shut == n:
            result.append(path.copy())
            return
        if start < n:
            path.append(1)
            backtrack(path, x + 1, y + 1, start + 1, shut, result)
            path.pop()
        if shut < start:
            path.append(-1)
            backtrack(path, x + 1, y - 1, start, shut + 1, result)
            path.pop()

    result = []
    backtrack([], 0, 0, 0, 0, result)
    return result

def catalan_number(n):
    return comb(2 * n, n) // (n + 1)

for n in range(1, 6):  # Test for n = 1 to 5
    dyck_paths = generate_dyck_paths(n)
    final_set = [ParkingFunction(dp) for dp in dyck_paths]
    print(f"Dyth Paths | n = {n} | Size = {catalan_number(n)}")
    run_homomesic_on_words(dyck_paths)
    print()

ValueError: [1, -1] is not a parking function